In [1]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np


In [2]:

X = pd.read_csv("credit_card_transactions.csv")

is_fraud = X[["is_fraud"]]

is_fraud.to_csv("is_fraud.csv", index=False)
#1296675 rows x 24 columns

print(X.head())

   Unnamed: 0 trans_date_trans_time            cc_num  \
0           0   2019-01-01 00:00:18  2703186189652095   
1           1   2019-01-01 00:00:44      630423337322   
2           2   2019-01-01 00:00:51    38859492057661   
3           3   2019-01-01 00:01:16  3534093764340240   
4           4   2019-01-01 00:03:06   375534208663984   

                             merchant       category     amt      first  \
0          fraud_Rippin, Kub and Mann       misc_net    4.97   Jennifer   
1     fraud_Heller, Gutmann and Zieme    grocery_pos  107.23  Stephanie   
2                fraud_Lind-Buckridge  entertainment  220.11     Edward   
3  fraud_Kutch, Hermiston and Farrell  gas_transport   45.00     Jeremy   
4                 fraud_Keeling-Crist       misc_pos   41.96      Tyler   

      last gender                        street  ...      long city_pop  \
0    Banks      F                561 Perry Cove  ...  -81.1781     3495   
1     Gill      F  43039 Riley Greens Suite 393  ... -11

In [3]:
import pytz
from datetime import datetime


time_column = 'trans_date_trans_time'
local_time_zone = 'Asia/Riyadh' 
def convert_to_utc(local_time_str):
    local_tz = pytz.timezone(local_time_zone)
    # Parse the string into a datetime object
    local_time = datetime.strptime(local_time_str, "%Y-%m-%d %H:%M:%S")
    # Localize and convert to UTC
    localized_time = local_tz.localize(local_time)
    utc_time = localized_time.astimezone(pytz.utc)
    return utc_time

# Apply the function to the dataset
X['utc_time'] = X['trans_date_trans_time'].apply(convert_to_utc)
X = X.drop(columns=['trans_date_trans_time'])
X = X.drop(columns=['is_fraud'])

# Save the updated dataset to a new CSV file
output_file = "updated_dataset_2.csv"
X.to_csv(output_file, index=False)

print(f"Converted times saved to {output_file}")

Converted times saved to updated_dataset_2.csv


In [4]:
Y = pd.read_csv("is_fraud.csv")

In [7]:
X = pd.read_csv("updated_dataset_2.csv")

In [18]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
columns_to_drop = ['Unnamed: 0','street', 'first', 'last','merch_zipcode']
X = X.drop(columns=columns_to_drop)

In [19]:
categorical_columns = ['merchant', 'category', 'gender', 'city',
                       'state', 'job']
label_encoders = {}  # Store encoders for potential inverse transformation
for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    X[col] = label_encoders[col].fit_transform(X[col])

In [24]:
from datetime import datetime

X['dob'] = pd.to_datetime(X['dob'], errors='coerce')  # Converts 'dob' to datetime
X['utc_time'] = pd.to_datetime(X['utc_time'], errors='coerce')  # Converts 'utc_time' to datetime

# Extract features from 'dob'
X['dob_year'] = X['dob'].dt.year
X['dob_month'] = X['dob'].dt.month
X['dob_day'] = X['dob'].dt.day

# Extract features from 'utc_time'
X['utc_hour'] = X['utc_time'].dt.hour

# Handle missing values (if necessary)
X['dob'].fillna(pd.Timestamp('2000-01-01'), inplace=True)  # Fill missing 'dob' with default date
X['utc_time'].fillna(method='ffill', inplace=True)  # Fill missing 'utc_time' with forward fill


In [27]:
columns_to_drop = [ 'dob', 'utc_time']
X = X.drop(columns=columns_to_drop)


In [31]:
columns_to_drop = [ 'trans_num']
X = X.drop(columns=columns_to_drop)


In [41]:
# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Scale the Data (important for KNN)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [42]:
# Feature Selection
X_train_selected = X_train
X_test_selected = X_test


In [44]:
import numpy as np

# Check for NaN values in the training set
print("NaN values in X_train_selected:", np.isnan(X_train_selected).sum())

# Check for NaN values in the testing set
print("NaN values in X_test_selected:", np.isnan(X_test_selected).sum())

NaN values in X_train_selected: 0
NaN values in X_test_selected: 0


In [ ]:
print(X.dtypes)

In [40]:
columns_to_drop = [ 'merch_zipcode']
X = X.drop(columns=columns_to_drop)


In [45]:
output_file = "updated_dataset_3.csv"
X.to_csv(output_file, index=False)

In [50]:
print(Y.keys)

<bound method NDFrame.keys of          is_fraud
0               0
1               0
2               0
3               0
4               0
...           ...
1296670         0
1296671         0
1296672         0
1296673         0
1296674         0

[1296675 rows x 1 columns]>


In [54]:
# Train KNN
knn = KNeighborsClassifier(n_neighbors=5)

# Reshape y_train and y_test
y_train = y_train.values.ravel()  # Convert to NumPy array and flatten
y_test = y_test.values.ravel()

knn.fit(X_train_selected, y_train)
print("Test Accuracy:", knn.score(X_test_selected, y_test))


AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [56]:
print("Shape of X:", X.shape)
print("Shape of Y:", Y.shape)

Shape of X: (1296675, 21)
Shape of Y: (1296675, 1)


In [57]:
Z = pd.read_csv("updated_dataset_3.csv")

In [58]:
print("Shape of Y:", Z.shape)

Shape of Y: (1296675, 21)


In [59]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Initialize the Logistic Regression model
clf = LogisticRegression(random_state=42, max_iter=1000)

# Train the model
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.993572020745368
Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00    257815
           1       0.00      0.00      0.00      1520

    accuracy                           0.99    259335
   macro avg       0.50      0.50      0.50    259335
weighted avg       0.99      0.99      0.99    259335



In [141]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import shuffle

# Assuming X (features) and y (labels)
X = X.sample(frac=1, random_state=42).reset_index(drop=True)
Y =Y.sample(frac=1, random_state=42).reset_index(drop=True)

In [142]:
print(Y)

         is_fraud
0               0
1               0
2               0
3               0
4               0
...           ...
1296670         0
1296671         0
1296672         0
1296673         0
1296674         0

[1296675 rows x 1 columns]


In [143]:
N = pd.read_csv("is_fraud.csv")
print(N)

         is_fraud
0               0
1               0
2               0
3               0
4               0
...           ...
1296670         0
1296671         0
1296672         0
1296673         0
1296674         0

[1296675 rows x 1 columns]


In [128]:
# Step 1: Split the dataset into Train, Validation, and Test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.4, random_state=42)  # 60% train, 40% temp
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 20% val, 20% test

print("Training Set:", X_train.shape, y_train.shape)
print("Validation Set:", X_val.shape, y_val.shape)
print("Test Set:", X_test.shape, y_test.shape)



Training Set: (778005, 21) (778005, 1)
Validation Set: (259335, 21) (259335, 1)
Test Set: (259335, 21) (259335, 1)


In [138]:
# Step 2: Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Step 3: Train the Logistic Regression model
clf = LogisticRegression(
    random_state=42, 
    solver='saga',  
    penalty='l2',   
    max_iter=1000,  
    tol=1e5,       
    C=1.0,        
    class_weight='balanced'
)
clf.fit(X_train, y_train)



C:\Users\HUAWEI\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42,
                   solver='saga', tol=100000.0)

In [130]:
# Step 4: Validate on the validation set
y_val_pred = clf.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)
print("Validation Classification Report:\n", classification_report(y_val, y_val_pred))



Validation Accuracy: 0.9166252144909094
Validation Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.92      0.96    257840
           1       0.05      0.74      0.09      1495

    accuracy                           0.92    259335
   macro avg       0.52      0.83      0.52    259335
weighted avg       0.99      0.92      0.95    259335



In [131]:
# Step 5: Test on the test set
y_test_pred = clf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)
print("Test Classification Report:\n", classification_report(y_test, y_test_pred))


Test Accuracy: 0.917454258006054
Test Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.92      0.96    257866
           1       0.05      0.72      0.09      1469

    accuracy                           0.92    259335
   macro avg       0.52      0.82      0.52    259335
weighted avg       0.99      0.92      0.95    259335



In [132]:
import joblib

# Save the model to a file
joblib.dump(clf, 'logistic_regression_model.pkl')

# Save the scaler as well to preprocess new data consistently
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [133]:
clf = joblib.load('logistic_regression_model.pkl')
scaler = joblib.load('scaler.pkl')

In [134]:

Z = pd.read_csv("updated_dataset_4.csv")
# Step 2: Take 10 rows from the dataset (e.g., the first 10 rows or a random sample)
X_new = Z.iloc[:10]  # First 10 rows

In [135]:
print(X_new.head())

   merchant  category     amt  gender  city  state    zip      lat      long  \
0       514         8    4.97       0   526     27  28654  36.0788  -81.1781   
1       241         4  107.23       0   612     47  99160  48.8878 -118.2105   
2       390         0  220.11       1   468     13  83252  42.1808 -112.2620   
3       360         2   45.00       1    84     26  59632  46.2306 -112.1138   
4       297         9   41.96       1   216     45  24433  38.4207  -79.4629   

   city_pop  ...   unix_time  merch_lat  merch_long  dob_year  dob_month  \
0      3495  ...  1325376018  36.011293  -82.048315      1988          3   
1       149  ...  1325376044  49.159047 -118.186462      1978          6   
2      4154  ...  1325376051  43.150704 -112.154481      1962          1   
3      1939  ...  1325376076  47.034331 -112.561071      1967          1   
4        99  ...  1325376186  38.674999  -78.632459      1986          3   

   dob_day  utc_hour  utc_minute  utc_second  service_provider

In [136]:

X_new_scaled = scaler.transform(X_new)  # Preprocess the new data

y_new_pred = clf.predict(X_new_scaled)

y_new_prob = clf.predict_proba(X_new_scaled)


In [137]:
print("Predictions:", y_new_pred)

# (Optional) Step 5: Predict probabilities
y_new_prob = clf.predict_proba(X_new_scaled)
print("Prediction Probabilities:", y_new_prob)

Predictions: [0 1 1 1 0 1 0 1 0 1]
Prediction Probabilities: [[0.71131796 0.28868204]
 [0.39837557 0.60162443]
 [0.17688079 0.82311921]
 [0.46653622 0.53346378]
 [0.72507633 0.27492367]
 [0.28966989 0.71033011]
 [0.56143298 0.43856702]
 [0.39873486 0.60126514]
 [0.64526515 0.35473485]
 [0.2208352  0.7791648 ]]


In [77]:
from itertools import combinations
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd

In [78]:
keys_list = list(X.columns)
print("Keys of X:", keys_list)

Keys of X: ['cc_num', 'merchant', 'category', 'amt', 'gender', 'city', 'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'unix_time', 'merch_lat', 'merch_long', 'dob_year', 'dob_month', 'dob_day', 'utc_hour', 'utc_minute', 'utc_second']


In [ ]:
X_keys: ['cc_num', 'merchant', 'category', 'amt', 'gender',
    'city', 'state', 'zip', 'lat', 'long', 'city_pop', 'job',
    'unix_time', 'merch_lat', 'merch_long', 'dob_year', 
    'dob_month', 'dob_day', 'utc_hour', 'utc_minute',
    'utc_second']

In [79]:
def get_service_provider(cc_number):
    cc_number = str(cc_number)  # Ensure the credit card number is a string
    if cc_number.startswith('4'):
        return 'Visa'
    elif 51 <= int(cc_number[:2]) <= 55 or 2221 <= int(cc_number[:4]) <= 2720:
        return 'MasterCard'
    elif cc_number.startswith('34') or cc_number.startswith('37'):
        return 'American Express'
    elif cc_number.startswith('6011') or cc_number.startswith('65') or \
         (644 <= int(cc_number[:3]) <= 649) or (622126 <= int(cc_number[:6]) <= 622925):
        return 'Discover'
    elif 3528 <= int(cc_number[:4]) <= 3589:
        return 'JCB'
    else:
        return 'Other'

In [80]:
X['service_provider'] = X['cc_num'].apply(get_service_provider)

columns_to_drop = [ 'cc_num']
X = X.drop(columns=columns_to_drop)


label_encoders = {}  # Store encoders for potential inverse transformation

label_encoders['service_provider'] = LabelEncoder()
X['service_provider'] = label_encoders['service_provider'].fit_transform(X['service_provider'])

In [81]:
output_file = "updated_dataset_4.csv"
X.to_csv(output_file, index=False)

In [ ]:
file_path = 'updated_dataset.csv'  # Replace with your dataset path
data = pd.read_csv(file_path)

# Define features (X) and target (Y)
X = data.drop(columns=['label_column'])  # Replace 'label_column' with your target column
Y = data['label_column']  # Replace 'label_column' with your target column

In [82]:
 list(X_train.columns)

['cc_num',
 'merchant',
 'category',
 'amt',
 'gender',
 'city',
 'state',
 'zip',
 'lat',
 'long',
 'city_pop',
 'job',
 'unix_time',
 'merch_lat',
 'merch_long',
 'dob_year',
 'dob_month',
 'dob_day',
 'utc_hour',
 'utc_minute',
 'utc_second']

In [85]:
y_train = y_train.values.ravel()  # Convert to NumPy array and flatten
y_test = y_test.values.ravel()


In [ ]:
# Greedy Forward Selection
selected_features = []
remaining_features = list(X_train.columns)
best_accuracy = 0

while len(selected_features) < 5 or x==25:
    x=+1
    best_feature = None
    for feature in remaining_features:
        # Test current feature with selected features
        current_features = selected_features + [feature]
        X_train_subset = X_train[current_features]
        X_test_subset = X_test[current_features]

        # Scale the data
        X_train_subset_scaled = scaler.fit_transform(X_train_subset)
        X_test_subset_scaled = scaler.transform(X_test_subset)

        # Train Logistic Regression
        model = LogisticRegression(max_iter=1000, random_state=42)
        model.fit(X_train_subset_scaled, y_train)

        # Evaluate
        y_pred = model.predict(X_test_subset_scaled)
        accuracy = accuracy_score(y_test, y_pred)

        # Check if this is the best feature to add
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_feature = feature

    # Add the best feature to the selected list
    if best_feature:
        selected_features.append(best_feature)
        remaining_features.remove(best_feature)

print("Selected Features:", selected_features)
print("Best Accuracy:", best_accuracy)


In [89]:
df = pd.read_csv('credit_card_transactions.csv')
print(df)

         Unnamed: 0 trans_date_trans_time               cc_num  \
0                 0   2019-01-01 00:00:18     2703186189652095   
1                 1   2019-01-01 00:00:44         630423337322   
2                 2   2019-01-01 00:00:51       38859492057661   
3                 3   2019-01-01 00:01:16     3534093764340240   
4                 4   2019-01-01 00:03:06      375534208663984   
...             ...                   ...                  ...   
1296670     1296670   2020-06-21 12:12:08       30263540414123   
1296671     1296671   2020-06-21 12:12:19     6011149206456997   
1296672     1296672   2020-06-21 12:12:32     3514865930894695   
1296673     1296673   2020-06-21 12:13:36     2720012583106919   
1296674     1296674   2020-06-21 12:13:37  4292902571056973207   

                                    merchant       category     amt  \
0                 fraud_Rippin, Kub and Mann       misc_net    4.97   
1            fraud_Heller, Gutmann and Zieme    grocery_pos  107.